In [5]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score

nltk.download('punkt')
nltk.download('stopwords')

# Charger les jeux de données
vrai_df = pd.read_csv('dataset/True.csv')
faux_df = pd.read_csv('dataset/Fake.csv')

print("les donnee qui ne sont pas des fake news")
print(vrai_df.head())

print("les donnee qui sont  des fake news")
print(faux_df.head())

# Ajouter des labels
vrai_df['label'] = 1
faux_df['label'] = 0

# Concaténer les jeux de données
data = pd.concat([vrai_df, faux_df])
print(data.head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


les donnee qui ne sont pas des fake news
                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017   
les donnee qui sont  des fake news
                      

In [7]:
# Convertir le texte en minuscules et tokeniser
data['text'] = data['text'].str.lower()
data['title'] = data['title'].str.lower()
data['text'] = data['text'].apply(nltk.word_tokenize)
data['title'] = data['title'].apply(nltk.word_tokenize)

# Supprimer les mots vides (stopwords)
stop_words = set(stopwords.words('english'))
data['text'] = data['text'].apply(lambda x: [mot for mot in x if mot not in stop_words])
data['title'] = data['title'].apply(lambda x: [mot for mot in x if mot not in stop_words])

In [9]:
# Initialiser le vectoriseur TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Appliquer TF-IDF sur les textes
text_tfidf = tfidf_vectorizer.fit_transform(data['text'].apply(lambda x: ' '.join(x)))
title_tfidf = tfidf_vectorizer.transform(data['title'].apply(lambda x: ' '.join(x)))

X = text_tfidf
y = data['label']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Afficher les formes des ensembles de données
print(f"Forme de X_train : {X_train.shape}")
print(f"Forme de X_test : {X_test.shape}")
print(f"Forme de y_train : {y_train.shape}")
print(f"Forme de y_test : {y_test.shape}")

Forme de X_train : (35918, 5000)
Forme de X_test : (8980, 5000)
Forme de y_train : (35918,)
Forme de y_test : (8980,)


In [13]:
#Naive Bayes Multinomial

In [15]:
naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(X_train, y_train)

y_pred = naive_bayes_model.predict(X_test)

# Évaluation
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Précision du Naive Bayes : {accuracy:.2f}")
print("Matrice de confusion Naive Bayes :")
print(confusion)
print("Rapport de classification Naive Bayes :")
print(classification_rep)

Précision du Naive Bayes : 0.94
Matrice de confusion Naive Bayes :
[[4410  240]
 [ 280 4050]]
Rapport de classification Naive Bayes :
              precision    recall  f1-score   support

           0       0.94      0.95      0.94      4650
           1       0.94      0.94      0.94      4330

    accuracy                           0.94      8980
   macro avg       0.94      0.94      0.94      8980
weighted avg       0.94      0.94      0.94      8980



In [17]:
#Arbre de Décision

In [19]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

def evaluate_decision_tree(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    confusion = confusion


In [20]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

def evaluate_decision_tree(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred)
    
    return accuracy, confusion, classification_rep

accuracy_tree, confusion_tree, classification_rep_tree = evaluate_decision_tree(decision_tree, X_test, y_test)

print(f"Précision de l'Arbre de Décision : {accuracy_tree:.2f}")
print("Matrice de confusion Arbre de Décision :")
print(confusion_tree)
print("Rapport de classification Arbre de Décision :")
print(classification_rep_tree)

Précision de l'Arbre de Décision : 1.00
Matrice de confusion Arbre de Décision :
[[4639   11]
 [  12 4318]]
Rapport de classification Arbre de Décision :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4650
           1       1.00      1.00      1.00      4330

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [23]:
#Classificateur Passif-Agressif

In [25]:
passive_aggressive = PassiveAggressiveClassifier()
passive_aggressive.fit(X_train, y_train)

y_pred = passive_aggressive.predict(X_test)

# Évaluation
accuracy_pa = accuracy_score(y_test, y_pred)
confusion_pa = confusion_matrix(y_test, y_pred)
classification_rep_pa = classification_report(y_test, y_pred)

print(f"Précision du Classificateur Passif-Agressif : {accuracy_pa:.2f}")
print("Matrice de confusion Classificateur Passif-Agressif :")
print(confusion_pa)
print("Rapport de classification Classificateur Passif-Agressif :")
print(classification_rep_pa)

Précision du Classificateur Passif-Agressif : 1.00
Matrice de confusion Classificateur Passif-Agressif :
[[4633   17]
 [  11 4319]]
Rapport de classification Classificateur Passif-Agressif :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4650
           1       1.00      1.00      1.00      4330

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [27]:
#Régression Logistique

In [29]:
logistic_regression = LogisticRegression(max_iter=1000)
logistic_regression.fit(X_train, y_train)

y_pred = logistic_regression.predict(X_test)

# Évaluation
accuracy_lr = accuracy_score(y_test, y_pred)
confusion_lr = confusion_matrix(y_test, y_pred)
classification_rep_lr = classification_report(y_test, y_pred)

print(f"Précision de la Régression Logistique : {accuracy_lr:.2f}")
print("Matrice de confusion Régression Logistique :")
print(confusion_lr)
print("Rapport de classification Régression Logistique :")
print(classification_rep_lr)

Précision de la Régression Logistique : 0.99
Matrice de confusion Régression Logistique :
[[4603   47]
 [  23 4307]]
Rapport de classification Régression Logistique :
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4650
           1       0.99      0.99      0.99      4330

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [31]:
#Forêt Aléatoire

In [33]:
random_forest = RandomForestClassifier()
random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

# Évaluation
accuracy_rf = accuracy_score(y_test, y_pred)
confusion_rf = confusion_matrix(y_test, y_pred)
classification_rep_rf = classification_report(y_test, y_pred)

print(f"Précision de la Forêt Aléatoire : {accuracy_rf:.2f}")
print("Matrice de confusion Forêt Aléatoire :")
print(confusion_rf)
print("Rapport de classification Forêt Aléatoire :")
print(classification_rep_rf)

Précision de la Forêt Aléatoire : 1.00
Matrice de confusion Forêt Aléatoire :
[[4640   10]
 [  11 4319]]
Rapport de classification Forêt Aléatoire :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4650
           1       1.00      1.00      1.00      4330

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [34]:
# Comparer les performances des modèles
algorithmes = ['Naive Bayes', 'Arbre de Décision', 'Classificateur Passif-Agressif', 'Régression Logistique', 'Forêt Aléatoire']
précisions = [accuracy, accuracy_tree, accuracy_pa, accuracy_lr, accuracy_rf]
rappels = [recall_score(y_test, naive_bayes_model.predict(X_test)),
           recall_score(y_test, decision_tree.predict(X_test)),
           recall_score(y_test, passive_aggressive.predict(X_test)),
           recall_score(y_test, logistic_regression.predict(X_test)),
           recall_score(y_test, random_forest.predict(X_test))]
f1_scores = [f1_score(y_test, naive_bayes_model.predict(X_test)),
             f1_score(y_test, decision_tree.predict(X_test)),
             f1_score(y_test, passive_aggressive.predict(X_test)),
             f1_score(y_test, logistic_regression.predict(X_test)),
             f1_score(y_test, random_forest.predict(X_test))]

# Afficher les résultats
for algo, precision, recall, f1 in zip(algorithmes, précisions, rappels, f1_scores):
    print(f"Algorithme : {algo}")
    print(f"  Précision : {precision:.2f}")
    print(f"  Rappel : {recall:.2f}")
    print(f"  F1 Score : {f1:.2f}")
    print()

Algorithme : Naive Bayes
  Précision : 0.94
  Rappel : 0.94
  F1 Score : 0.94

Algorithme : Arbre de Décision
  Précision : 1.00
  Rappel : 1.00
  F1 Score : 1.00

Algorithme : Classificateur Passif-Agressif
  Précision : 1.00
  Rappel : 1.00
  F1 Score : 1.00

Algorithme : Régression Logistique
  Précision : 0.99
  Rappel : 0.99
  F1 Score : 0.99

Algorithme : Forêt Aléatoire
  Précision : 1.00
  Rappel : 1.00
  F1 Score : 1.00



In [37]:
def prédire_titre(titre_texte):
    titre_tfidf = tfidf_vectorizer.transform([titre_texte])
    prédiction = naive_bayes_model.predict(titre_tfidf)
    return "Vrai" if prédiction[0] == 1 else "Faux"

# Exemple de prédiction
exemple_titre = "Le changement climatique s'accélère, les scientifiques alertent sur une catastrophe imminente"
print(f"Prédiction pour le titre '{exemple_titre}' : {prédire_titre(exemple_titre)}")


Prédiction pour le titre 'Le changement climatique s'accélère, les scientifiques alertent sur une catastrophe imminente' : Vrai


In [39]:

# Exemple de prédiction
exemple_titre = "La NASA confirme que la Terre sera plongée dans 15 jours de noir total en raison d'une tempête solaire"
print(f"Prédiction pour le titre '{exemple_titre}' : {prédire_titre(exemple_titre)}")

Prédiction pour le titre 'La NASA confirme que la Terre sera plongée dans 15 jours de noir total en raison d'une tempête solaire' : Faux


In [41]:
# Exemple de prédiction
exemple_titre = "Un astéroïde géant détruira la Terre dans un mois, préviennent les experts"
print(f"Prédiction pour le titre '{exemple_titre}' : {prédire_titre(exemple_titre)}")

Prédiction pour le titre 'Un astéroïde géant détruira la Terre dans un mois, préviennent les experts' : Faux


In [43]:
# Exemple de prédiction
exemple_titre = "Avec l’hiver et le printemps froids que l’on vient de subir, on parle encore de réchauffement climatique !? "
print(f"Prédiction pour le titre '{exemple_titre}' : {prédire_titre(exemple_titre)}")

Prédiction pour le titre 'Avec l’hiver et le printemps froids que l’on vient de subir, on parle encore de réchauffement climatique !? ' : Faux


In [45]:
# Exemple de prédiction
exemple_titre = "Les Extraterrestres Ont Débarqué en Secret dans le Désert du Nevada "
print(f"Prédiction pour le titre '{exemple_titre}' : {prédire_titre(exemple_titre)}")

Prédiction pour le titre 'Les Extraterrestres Ont Débarqué en Secret dans le Désert du Nevada ' : Faux


In [47]:
# Exemple de prédiction
exemple_titre = "Un Remède Miracle Découvert pour Guérir Toutes les Maladies en Une Semaine"
print(f"Prédiction pour le titre '{exemple_titre}' : {prédire_titre(exemple_titre)}")

Prédiction pour le titre 'Un Remède Miracle Découvert pour Guérir Toutes les Maladies en Une Semaine' : Faux


In [49]:
# Exemple de prédiction
exemple_titre = "Des Espèces Maritimes Géantes Découvertes dans le Lac Tahoe "
print(f"Prédiction pour le titre '{exemple_titre}' : {prédire_titre(exemple_titre)}")

Prédiction pour le titre 'Des Espèces Maritimes Géantes Découvertes dans le Lac Tahoe ' : Vrai


In [51]:
# Exemple de prédiction
exemple_titre = "Les Célébrités Sont Entrain de Quitter la Terre pour Une Nouvelle Planète Habitable"
print(f"Prédiction pour le titre '{exemple_titre}' : {prédire_titre(exemple_titre)}")

Prédiction pour le titre 'Les Célébrités Sont Entrain de Quitter la Terre pour Une Nouvelle Planète Habitable' : Faux


In [53]:
# Exemple de prédiction
exemple_titre = "Les États-Unis Annoncent des Sanctions Économiques Contre un Nouveau Pays "
print(f"Prédiction pour le titre '{exemple_titre}' : {prédire_titre(exemple_titre)}")

Prédiction pour le titre 'Les États-Unis Annoncent des Sanctions Économiques Contre un Nouveau Pays ' : Vrai


In [55]:
# Exemple de prédiction
exemple_titre = "L'Organisation Mondiale de la Santé Publie un Nouveau Rapport sur la COVID-19"
print(f"Prédiction pour le titre '{exemple_titre}' : {prédire_titre(exemple_titre)}")

Prédiction pour le titre 'L'Organisation Mondiale de la Santé Publie un Nouveau Rapport sur la COVID-19' : Faux


In [57]:
# Exemple de prédiction
exemple_titre = "Des Études Récentes Montrent une Réduction de l'Émission de Gaz à Effet de Serre en 2023"
print(f"Prédiction pour le titre '{exemple_titre}' : {prédire_titre(exemple_titre)}")

Prédiction pour le titre 'Des Études Récentes Montrent une Réduction de l'Émission de Gaz à Effet de Serre en 2023' : Vrai


In [59]:
# Exemple de prédiction
exemple_titre = "Un Accord International Signé pour Lutte Contre la Déforestation en Amazonie"
print(f"Prédiction pour le titre '{exemple_titre}' : {prédire_titre(exemple_titre)}")

Prédiction pour le titre 'Un Accord International Signé pour Lutte Contre la Déforestation en Amazonie' : Vrai


In [61]:
# Exemple de prédiction
exemple_titre = "La NASA Annonce la Découverte d'une Nouvelle Exoplanète Potentiellement Habitable"
print(f"Prédiction pour le titre '{exemple_titre}' : {prédire_titre(exemple_titre)}")

Prédiction pour le titre 'La NASA Annonce la Découverte d'une Nouvelle Exoplanète Potentiellement Habitable' : Faux


In [63]:
pip install joblib


Note: you may need to restart the kernel to use updated packages.


In [64]:
import joblib

# Enregistrer le modèle Naive Bayes
joblib.dump(naive_bayes_model, 'model_fake_news.pkl')
print("Modèle Naive Bayes enregistré sous 'model_fake_news.pkl'")



Modèle Naive Bayes enregistré sous 'model_fake_news.pkl'


In [67]:
import joblib
# Sauvegarder le vectoriseur TF-IDF
joblib.dump(tfidf_vectorizer, 'vectorizer.pkl')
print("le modele vectorizer est enregistre avec succes")

le modele vectorizer est enregistre avec succes


In [69]:
# Charger le modèle Naive Bayes
naive_bayes_model_loaded = joblib.load('model_fake_news.pkl')

# Charger le vectoriseur TF-IDF
tfidf_vectorizer_loaded = joblib.load('vectorizer.pkl')

# Vérifier la prédiction avec le modèle chargé
exemple_titre = "Le changement climatique s'accélère, les scientifiques alertent sur une catastrophe imminente"
titre_tfidf = tfidf_vectorizer_loaded.transform([exemple_titre])
prédiction = naive_bayes_model_loaded.predict(titre_tfidf)

# Afficher la prédiction
print(f"Prédiction pour le titre '{exemple_titre}' avec le modèle chargé : {'Vrai' if prédiction[0] == 1 else 'Faux'}")



Prédiction pour le titre 'Le changement climatique s'accélère, les scientifiques alertent sur une catastrophe imminente' avec le modèle chargé : Vrai
